# REST API 활용
### 1️⃣ 다중 API 호출 패턴
여러 개의 엔드포인트를 활용하여 데이터를 조합할 때는 다음과 같은 패턴이 사용된다.

#### ① 순차적 API 호출
한 API 호출의 결과를 기반으로 다음 API를 호출하는 방식.
예제: 사용자 정보를 조회한 후, 해당 사용자의 주문 정보를 가져오기

In [2]:
import requests

# 1. 예)사용자 정보 조회
# user_response = requests.get("https://api.example.com/users/123")
user_response = requests.get("https://jsonplaceholder.typicode.com/posts/1")
user_data = user_response.json()

# 2. 예) 사용자의 주문 정보 조회
# orders_response = requests.get(f"https://api.example.com/orders?user_id={user_data['id']}")
orders_response = requests.get(f"https://jsonplaceholder.typicode.com/posts/1?user_id={user_data['id']}") # 실습용 가짜 데이터임
orders_data = orders_response.json()

print(orders_data)

{'userId': 1, 'id': 1, 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}


#### ② 병렬 API 호출
여러 개의 API를 동시에 호출하여 성능을 최적화하는 방식.
예제: asyncio와 aiohttp를 사용한 비동기 API 호출
- Jupyter Notebook 환경에서는 nest_asyncio를 적용하면 asyncio.run()을 사용할 수 있습니다.

In [6]:
import nest_asyncio
import asyncio
import aiohttp

nest_asyncio.apply()  # Jupyter 환경에서 asyncio 실행 가능하게 설정

async def fetch_data(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            return await response.json()

async def main():
    # user_url = "https://api.example.com/users/123"
    # orders_url = "https://api.example.com/orders?user_id=123"

    user_url = "https://jsonplaceholder.typicode.com/posts/1"
    orders_url = "https://jsonplaceholder.typicode.com/posts/1?user_id={user_data['id']}"
    
    # 두 개의 API를 동시에 호출
    user_data, orders_data = await asyncio.gather(
        fetch_data(user_url),
        fetch_data(orders_url)
    )

    print(user_data, orders_data)

asyncio.run(main())


{'userId': 1, 'id': 1, 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'} {'userId': 1, 'id': 1, 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}


### 2️⃣ 데이터 가공
API로 가져온 데이터는 대부분 정제 및 가공 과정이 필요하다.

(1) JSON 데이터 가공
JSON 데이터에서 특정 필드만 추출하거나 변환하는 방법.

In [7]:
import json

# 예제 JSON 데이터
response_data = '''
{
    "user": {
        "id": 123,
        "name": "Jane Doe",
        "email": "jane.doe@example.com"
    },
    "orders": [
        {"id": 1, "item": "Laptop", "price": 1200},
        {"id": 2, "item": "Mouse", "price": 50}
    ]
}
'''

# JSON 파싱
data = json.loads(response_data)

# 필요한 데이터만 가공
user_info = {
    "id": data["user"]["id"],
    "name": data["user"]["name"],
    "total_orders": len(data["orders"])
}

print(user_info)

{'id': 123, 'name': 'Jane Doe', 'total_orders': 2}


(2) Pandas를 활용한 데이터 정리
- REST API 응답 데이터를 Pandas 데이터프레임으로 변환하여 분석.

In [4]:
import pandas as pd

# API 응답 데이터
orders = [
    {"id": 1, "item": "Laptop", "price": 1200},
    {"id": 2, "item": "Mouse", "price": 50},
    {"id": 3, "item": "Keyboard", "price": 100}
]

# 데이터프레임 변환
df = pd.DataFrame(orders)

# 총 가격 계산
df["total_price"] = df["price"] * 1.1  # 부가세 10% 적용

df

,id,item,price,total_price
0,1,Laptop,1200,1320.0
1,2,Mouse,50,55.0
2,3,Keyboard,100,110.0


#### (3) API 데이터 필터링 및 정렬
- 특정 조건을 만족하는 데이터만 필터링하고 정렬할 수도 있다.

In [7]:
# price가 100달러 이상인 주문만 필터링 후 가격 기준으로 정렬
filtered_df = df[df["price"] >= 100].sort_values(by="price", ascending=False)

filtered_df

,id,item,price,total_price
0,1,Laptop,1200,1320.0
2,3,Keyboard,100,110.0


### 3️⃣ REST API 데이터 활용 예제
#### (1) 외부 API 활용 예제 - OpenWeather API
- 날씨 데이터를 가져와서 현재 온도를 출력하는 예제.

In [12]:
import requests

API_KEY = "your_api_key"  # 실제 https://openweathermap.org/ 사이트에 무료/유료 서비스 구독해야 사용가능하다
city = "Seoul"
url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"

response = requests.get(url)
weather_data = response.json()

# 필요한 데이터만 출력 : 이 소스는 API Key 미 사용으로 결과 데이터 얻을 수 없음
# current_temp = weather_data["main"]["temp"]
# print(f"{city}의 현재 온도: {current_temp}°C")

weather_data

{'cod': 401,
 'message': 'Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'}

#### (2) 웹훅(Webhook) 활용
- 특정 이벤트 발생 시 API 호출을 자동화
예: 사용자가 가입하면 이메일 API를 호출하여 환영 이메일 발송

In [14]:
import requests

# 이메일 API 엔드포인트 (예제) --> 실제 이메일 API가 아니라 실행 안됨
email_api_url = "https://api.emailservice.com/send"

# 이메일 전송 데이터
payload = {
    "to": "jane.doe@example.com",
    "subject": "Welcome!",
    "body": "Jane, 환영합니다!"
}

# POST 요청 보내기
# response = requests.post(email_api_url, json=payload)
# print(response.status_code, response.json())

In [ ]:
import requests

# SendGrid API 엔드포인트
email_api_url = "https://api.sendgrid.com/v3/mail/send"

# SendGrid API 키 (환경 변수로 관리 권장)
API_KEY = "YOUR_SENDGRID_API_KEY"

# 이메일 전송 데이터
payload = {
    "personalizations": [
        {
            "to": [{"email": "jane.doe@example.com"}],
            "subject": "Welcome!"
        }
    ],
    "from": {"email": "your_email@example.com"},
    "content": [{"type": "text/plain", "value": "Jane, 환영합니다!"}]
}

# 헤더 설정
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# POST 요청 보내기
response = requests.post(email_api_url, headers=headers, json=payload)

# 응답 출력
print(response.status_code, response.json() if response.status_code != 202 else "Email Sent Successfully!")

In [ ]:
# import smtplib
# from email.mime.text import MIMEText

# # SMTP 서버 설정
# SMTP_SERVER = "smtp.gmail.com"
# SMTP_PORT = 587
# EMAIL_ADDRESS = "your_email@gmail.com"  # Gmail 주소
# EMAIL_PASSWORD = "your_app_password"   # 앱 비밀번호 사용 (일반 비밀번호 X)

# # 이메일 내용 설정
# msg = MIMEText("Jane, 환영합니다!")
# msg["Subject"] = "Welcome!"
# msg["From"] = EMAIL_ADDRESS
# msg["To"] = "jane.doe@example.com"

# # SMTP 서버 연결 및 이메일 전송
# with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
#     server.starttls()
#     server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
#     server.sendmail(EMAIL_ADDRESS, "jane.doe@example.com", msg.as_string())

# print("Email Sent Successfully!")